# ANN variables testing

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


def preprocess_clean_data(dane: pd.DataFrame, y_col: str):
    dane = dane.rename(columns={"#Layovers": "Num_Layovers", "Price [PLN]": "Price"})

    # Przekształcenie dat
    if "Flight_date" in dane.columns:
        dane["Flight_date"] = pd.to_datetime(dane["Flight_date"], errors="coerce")
    if "Extraction_Time" in dane.columns:
        dane["Extraction_Time"] = pd.to_datetime(
            dane["Extraction_Time"].str.split(" ").apply(lambda x: x[0]),
            dayfirst=True, errors="coerce"
        )
    dane["Ticket_class"] = dane["Ticket_class"].map({"Ekonomiczna": 0, "Biznes": 1})
    # Usunięcie zbędnych kolumn
    kolumny_do_usuniecia = [
        "Extraction_Time", "Flight_date", "arr_city", "dep_city",
        "Departure_airport_name", "Destination_airport_name",
        "layover_airport", "ujemne", "low_cost1", "low_cost2",
        "Departure_airport_code", "Destination_airport_code",
        "Flight_weekday", "Extraction_Weekday",
        "Airline1","Airline2","Is_-2"
    ]
    dane.drop(columns=kolumny_do_usuniecia, inplace=True, errors='ignore')

    # Oddzielenie celu od cech
    y = dane[y_col].copy()
    X = dane.drop(columns=[y_col])

    # Konwersja kolumn tekstowych do liczb, jeśli trzeba
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = pd.to_numeric(X[col], errors='coerce')

    # Uzupełnianie braków
    X.fillna(0, inplace=True)
    y.fillna(y.mean(), inplace=True)

    return X, y



# **Main code:**


In [5]:

import pandas as pd
from perceptron_nn import MultiLayerPerceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from glob import glob
# === Parametry eksperymentu ===
num_repetitions = 5
target_col = "Price"

# === Wczytanie i wstępne przetworzenie danych ===
df = pd.read_excel("loty_clean.xlsx")
X_full, y_full = preprocess_clean_data(df.copy(), y_col=target_col)

# Lista kolumn do testowania (po preprocessingu)
testable_columns = X_full.columns.tolist()

# Parametry sieci (bazowe)
baseline_params = {
    "num_layers": [15],
    "learning_rate": 0.1,
    "activation_function": "relu",
    "num_epochs": 2000
}

# Lista wyników
results = []

# === Pętla po kolumnach ===
for col in testable_columns:
    print(f"\n🧪 Testuję bez kolumny: {col}")

    X_temp = X_full.drop(columns=[col])

    try:
        # Podział na train/val/test
        X_train_np, X_temp_np, y_train_np, y_temp_np = train_test_split(X_temp, y_full, test_size=0.3, random_state=42)
        X_val_np, X_test_np, y_val_np, y_test_np = train_test_split(X_temp_np, y_temp_np, test_size=0.5, random_state=42)
    except Exception as e:
        print(f"❌ Błąd przy podziale danych (kolumna {col}): {e}")
        continue

    # Miejsce na metryki z powtórzeń
    train_mse, val_mse, test_mse, test_mae, test_mape, test_r2 = [], [], [], [], [], []

    for i in range(num_repetitions):
        print(f"   🔁 Powtórzenie {i+1}/{num_repetitions}")
        try:
            mlp = MultiLayerPerceptron(
                num_inputs=X_train_np.shape[1],
                num_layers=baseline_params["num_layers"],
                learning_rate=baseline_params["learning_rate"],
                activation_function=baseline_params["activation_function"]
            )

            mlp.fit(
            X=X_train_np,
            y=y_train_np.values.reshape(-1, 1),
            X_val=X_val_np,
            y_val=y_val_np.values.reshape(-1, 1),
                num_epochs=baseline_params["num_epochs"],
                verbose=False
            )

            y_pred = mlp.predict(X_test_np)
            hist = mlp.history

            train_mse.append(hist["train_mse"])
            val_mse.append(hist.get("val_mse", np.nan))
            test_mse.append(mean_squared_error(y_test_np, y_pred))
            test_mae.append(mean_absolute_error(y_test_np, y_pred))
            test_mape.append(mean_absolute_percentage_error(y_test_np, y_pred))
            test_r2.append(r2_score(y_test_np, y_pred))

        except Exception as e:
            print(f"❌ Błąd treningu dla kolumny {col}: {e}")
            continue

    # Zapisz średnie wyniki dla danej kolumny
    results.append({
        "usunięta_kolumna": col,
        "train_MSE": np.mean(train_mse),
        "val_MSE": np.mean(val_mse),
        "test_MSE": np.mean(test_mse),
        "test_MAE": np.mean(test_mae),
        "test_MAPE": np.mean(test_mape),
        "test_R2": np.mean(test_r2)
    })

# === Zapis wyników ===
results_df = pd.DataFrame(results)
filename = f"dobor_zmiennych_nn_{len(glob('dobor_zmiennych_nn_*.xlsx')) + 1}.xlsx"
results_df.to_excel(filename, index=False)
print(f"\n✅ Wyniki zapisane do pliku: {filename}")




🧪 Testuję bez kolumny: Departure_time
   🔁 Powtórzenie 1/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście(1)
Early stopping na epoce 274/2000. 50 epok bez poprawy. Najlepszy val_MSE: 274874.8481
   🔁 Powtórzenie 2/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście(1)
Early stopping na epoce 277/2000. 50 epok bez poprawy. Najlepszy val_MSE: 273435.5564
   🔁 Powtórzenie 3/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście(1)
Early stopping na epoce 282/2000. 50 epok bez poprawy. Najlepszy val_MSE: 264883.5316
   🔁 Powtórzenie 4/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście(1)
Early stopping na epoce 260/2000. 50 epok bez poprawy. Najlepszy val_MSE: 271441.2700
   🔁 Powtórzenie 5/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście(1)
Early stopping na epoce 274/2000. 50 epok bez poprawy. Najlepszy val_MSE: 271751.4831

🧪 Testuję bez kolumny: Arrival_time
   🔁 Powtórzenie 1/5
Architektura sieci: Wejścia(8) -> Warstwa_1(15) -> Wyjście

## Interpretacja:

-  `Ticket_class` zawiera dużo informacji istotnych dla przewidywania ceny biletu, ponieważ jej brak powoduje największy spadek jakości predykcji
- `Cabin_bag`, `Days_to_departure`, `Num_Layovers` wskazują, że ich brak ogranicza zdolność sieci do przewidywania zmienności ceny
- `Checked_bag`, `Arrival_time` oraz `Departure_time` to mniej istotne zmienne, $R^2$ wciąż pozostaje na poziomie powyżej 0.75

Warto zauważyć, że `Flight_time`, `Arrival_time` oraz `Departure_time` dają bardzo podobne wyniki, oznacza to że te zmienne mogą być z sobą mocno skorelowaną (wynika to też
z czystej logiki).